# Assignment: Notebooks
### *Same text as lab.md/assignment.md* 

Using the notebooks associated to this lesson (either the PyTorch or the TensorFlow version), rerun them using your own dataset, perhaps one from Kaggle, used with attribution. Rewrite the notebook to underline your own findings. Try a different kind of dataset and document your findings, using text such as [this Kaggle competition dataset about weather tweets](https://www.kaggle.com/competitions/crowdflower-weather-twitter/data?select=train.csv).

In [3]:
import torch
from torchtext import *
from kalebtorchnlp import *
from torch.utils.data import DataLoader, Dataset
import portalocker
import numpy as np
from torch.utils.data import DataLoader


/home/kaleb/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
# def preprocess(df, target, test_size=0.2, random_state=42, printdf=False):
#     if target not in df.columns:
#         print(f"Column '{target}' does not exist in the dataframe")

#     df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function']
#     df['text'] = df['text'].str.lower()
#     df.drop(['title', 'location', 'department', 'company_profile', 'description', 'requirements', 'benefits', 'employment_type', 'required_experience', 'required_education', 'industry', 'function'], axis=1, inplace=True)


#     if printdf == True:
#         print(df.columns)
#         print(df.head())
#         print(df[target])
    
#     # train_df, test_df = train_test_split(df, test_size=test_size, random_state=random_state)
#     if target not in df.columns:
#         print(f"Column '{target}' does not exist in the dataframe")
#     train_df, test_df = train_test_split(df['text'], test_size = test_size , random_state = random_state)
#     train_labels , test_labels = train_test_split(df[target], test_size = test_size , random_state = random_state)



#     train_dataset = CSVDataset(train_df, target)
#     test_dataset = CSVDataset(test_df, target)

#     return train_dataset, test_dataset, train_labels, test_labels

# datasetDir = 'data/datasets/fake_job_postings.csv'
# target = "fraudulent"
# df = pd.read_csv(datasetDir)

# train_dataset, test_dataset = preprocess(df, target=target, printdf=True)

# print(train_dataset[0])

# batch_size = 32

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [8]:
class CSVDataset(Dataset):
    def __init__(self, dataframe, target_column):
        # print(dataframe.columns)
        # print(dataframe[target_column])
        self.dataframe = dataframe
        self.target_column = target_column
        self.features = dataframe.drop([target_column],axis=1 )
        self.targets = dataframe[target_column]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        features = self.features.iloc[idx].values.tolist()
        target = self.targets.iloc[idx]
        return features, target


In [9]:
def preprocess(df, target, test_size=0.2, random_state=42, printdf=False):
    if target not in df.columns:
        raise KeyError(f"Column '{target}' does not exist in the dataframe")

    # Combine text columns
    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function']
    df['text'] = df['text'].str.lower()
    
    # Drop original text columns
    df.drop(['title', 'location', 'department', 'company_profile', 'description', 'requirements', 'benefits', 'employment_type', 'required_experience', 'required_education', 'industry', 'function'], axis=1, inplace=True)

    if printdf == True:
        print("Columns in dataframe after preprocessing:", df.columns)
        print("First few rows of the dataframe:")
        print(df.head())
        print("Target column data:")
        print(df[target].head())
    
    # Assuming CSVDataset takes text and labels separately
    prep_df = CSVDataset(df, target)

    # Split the data
    train_df, test_df, train_labels, test_labels = train_test_split(prep_df.features, prep_df.target, test_size=test_size, random_state=random_state)
    if printdf == True:
        print(f"Train data shape: {train_df.shape}")
        print(f"Test data shape: {test_df.shape}")
        print(f"Train labels shape: {train_labels.shape}")
        print(f"Test labels shape: {test_labels.shape}")    



    return train_dataset, test_dataset

datasetDir = 'data/datasets/fake_job_postings.csv'
target = 'fraudulent'
df = pd.read_csv(datasetDir)

# Print column names and first few rows for debugging
# print("Columns in dataframe:", df.columns)
# print("First few rows of the dataframe:")
# print(df.head())

train_dataset, test_dataset, train_labels, test_labels = preprocess(df, target=target, printdf=True)


Columns in dataframe after preprocessing: Index(['job_id', 'salary_range', 'telecommuting', 'has_company_logo',
       'has_questions', 'fraudulent', 'text'],
      dtype='object')
First few rows of the dataframe:
   job_id salary_range  telecommuting  has_company_logo  has_questions  \
0       1          NaN              0                 1              0   
1       2          NaN              0                 1              0   
2       3          NaN              0                 1              0   
3       4          NaN              0                 1              0   
4       5          NaN              0                 1              1   

   fraudulent                                               text  
0           0                                                NaN  
1           0                                                NaN  
2           0                                                NaN  
3           0  account executive - washington dc us, dc, wash...  
4     

AttributeError: 'CSVDataset' object has no attribute 'target'

In [ ]:
dataDir = 'data/datasets'
datasetDir = dataDir + '/fake_job_postings.csv'

classes = ['job_id', 'title', 'location', 'department', 'salary_range', 'company_profile', 'description', 'requirements', 'benefits', 'telecommuting']

train_dataset, test_dataset, vocab = load_dataset(datasetDir, target, classes)
vocab_size = len(vocab)

# df = pd.read_csv(datasetDir)

# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
# vocab = torchtext.vocab.vocab(collections.Counter(), min_freq=1)
# vocab_size = len(vocab)

In [ ]:
train_df

In [ ]:
def preprocess(df, target, test_size=0.2, random_state=42, printdf=False):
    if target not in df.columns:
        print(f"Column '{target}' does not exist in the dataframe")

    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function']
    df['text'] = df['text'].str.lower()
    df.drop(['title', 'location', 'department', 'company_profile', 'description', 'requirements', 'benefits', 'employment_type', 'required_experience', 'required_education', 'industry', 'function'], axis=1, inplace=True)


    if printdf == True:
        print(df.columns)
        print(df.head())
        print(df[target])
    
    # train_df, test_df = train_test_split(df, test_size=test_size, random_state=random_state)
    if target not in df.columns:
        print(f"Column '{target}' does not exist in the dataframe")

    prep_df = CSVDataset(df, target)
    
    train_df, test_df = train_test_split(prep_df, test_size = test_size , random_state = random_state)
    # train_labels , test_labels = train_test_split(df[target], test_size = test_size , random_state = random_state)



    train_dataset = CSVDataset(train_df, target)
    test_dataset = CSVDataset(test_df, target)

    return train_dataset, test_dataset

batch_size = 32
target_column = 'fraudulent'

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f"Train length: {len(train_dataloader)}. Test length: {len(test_dataloader)}.")

NameError: name 'DataLoader' is not defined

# Simple RNN Classifier

In [ ]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

In [ ]:
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)